In [47]:
import copy
from pathlib import Path
import warnings
import numpy as np

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor

from pytorch_forecasting import TimeSeriesDataSet, TemporalFusionTransformer

import pandas as pd
# load data
data = pd.read_csv('../datasets/daily-min-temperatures-australia.csv')

# define dataset
max_encode_length = 36
max_prediction_length = 6
training_cutoff = "1990-05-13"  # day for cutoff


In [20]:
data.Date = pd.to_datetime(data.Date)

In [14]:
data['group'] = '0'

In [32]:
data["time_idx"] = data['Date'] - data['Date'].min()
data["time_idx"] = data['time_idx'].apply(lambda x: int(x.days))

In [81]:


max_prediction_length = 6
max_encoder_length = 24
training_cutoff = data["time_idx"].max() - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    time_idx='time_idx',
    group_ids=['group'],
    target='Temp',
    allow_missings=True,
    add_relative_time_idx=True,
    add_target_scales=True,
    add_encoder_length=True,
    
)


validation = TimeSeriesDataSet.from_dataset(training, data, min_prediction_idx=training.index.time.max() + 1, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=2)
val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size, num_workers=2)


early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=10, verbose=True, mode="min")
lr_logger = LearningRateMonitor()
trainer = pl.Trainer(
    max_epochs=100,
    gpus=1,
    gradient_clip_val=0.1,
    limit_train_batches=30,
    callbacks=[lr_logger, early_stop_callback],
)


tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=0.03,
    hidden_size=32,
    attention_head_size=1,
    dropout=0.1,
    hidden_continuous_size=16,
    output_size=11,
    loss=QuantileLoss(),
    log_interval=2,
    reduce_on_plateau_patience=4
)
print(f"Number of parameters in network: {tft.size()/1e3:.1f}k")


trainer.fit(
    tft, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | QuantileLoss                    | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 0     
3  | prescalers                         | ModuleDict                      | 128   
4  | static_variable_selection          | VariableSelectionNetwork        | 5 K   
5  | encoder_variable_selection         | VariableSelectionNetwork        | 1 K   
6  | decoder_variable_selection         | VariableSelectionNetwork        | 1 K   
7  | static_context_variable_selection  | GatedResidualNetwork            | 4 K   
8  | static_context_initial_hidden_lstm | GatedR

Number of parameters in network: 64.0k


C:\Users\DEJOKUR\AppData\Local\Continuum\anaconda3\lib\site-packages\pytorch_lightning\utilities\distributed.py:45: DeprecationWarning: Displayed epoch numbers by `EarlyStopping` start from "1" until v0.6.x, but will start from "0" in v0.8.0.
  warnings.warn(*args, **kwargs)
Epoch 00014: early stopping triggered.


1

In [ ]:
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

# create study
study = optimize_hyperparameters(
    train_dataloader,
    val_dataloader,
    model_path="optuna_test",
    n_trials=200,
    max_epochs=50,
    gradient_clip_val_range=(0.01, 1.0),
    hidden_size_range=(8, 128),
    hidden_continuous_size_range=(8, 128),
    attention_head_size_range=(1, 4),
    learning_rate_range=(0.001, 0.1),
    dropout_range=(0.1, 0.3),
    trainer_kwargs=dict(limit_train_batches=30),
    reduce_on_plateau_patience=4,
    use_learning_rate_finder=False,  # use Optuna to find ideal learning rate or use in-built learning rate finder
)

# save study results - also we can resume tuning at a later point in time
with open("test_study.pkl", "wb") as fout:
    pickle.dump(study, fout)

# show best hyperparameters
print(study.best_trial.params)

In [84]:
# load the best model according to the validation loss
# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_tft = TemporalFusionTransformer.load_from_checkpoint(best_model_path)

In [85]:
# calcualte mean absolute error on validation set
actuals = torch.cat([y for x, y in iter(val_dataloader)])
predictions = best_tft.predict(val_dataloader)
(actuals - predictions).abs().mean()

tensor(1.9004)

In [90]:
c = 0
for x, y in iter(val_dataloader):
    c = c +1 
#     print(x,y)
    print(c)

1


In [88]:
actuals.shape

torch.Size([36, 1])

In [77]:
predictions.shape

torch.Size([36, 1])

In [87]:
predictions

tensor([[13.2972],
        [13.2464],
        [12.9194],
        [12.9578],
        [12.9507],
        [12.8783],
        [12.8042],
        [12.8507],
        [12.9641],
        [13.1795],
        [13.4725],
        [13.5461],
        [13.5539],
        [13.5574],
        [13.5907],
        [13.5751],
        [13.6498],
        [13.6959],
        [13.6183],
        [13.5484],
        [13.5550],
        [13.5976],
        [13.7562],
        [13.9728],
        [14.1239],
        [14.3420],
        [14.2797],
        [14.3112],
        [14.4159],
        [14.3840],
        [14.3808],
        [14.4358],
        [14.5904],
        [14.5546],
        [14.5155],
        [14.6134]])